In [1]:
import torch
from torch_geometric.data import InMemoryDataset
import numpy as np
from torch_geometric.data import Data
import scipy.io

def f1_calculation(y,patient_results):
    y_pred = np.zeros(len(y))
    for i,label in enumerate(patient_results) :
        if label == 1 :
            y_pred[i]=y[i]
        else :
            y_pred[i] = 1-y[i]
    return f1_score(y,y_pred)

class PairData(Data):
    def __init__(self, edge_index_s=None, x_s=None, edge_index_t=None, x_t=None, y=None):
        super().__init__()
        self.edge_index_s = edge_index_s
        self.x_s = x_s
        self.edge_index_t = edge_index_t
        self.x_t = x_t
        self.y = y
        
    def __inc__(self, key, value, *args, **kwargs):
        if key == 'edge_index_s':
            return self.x_s.size(0)
        if key == 'edge_index_t':
            return self.x_t.size(0)
        else:
            return super().__inc__(key, value, *args, **kwargs)
        

class YooChooseBinaryDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(YooChooseBinaryDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []
    @property
    def processed_file_names(self):
        return ['C:\\Users\\User\\yoochoose-data\\discrepancy-capa-snet-nw-capa-averaging-smooth-norm1_3-adsci.dataset']

    def download(self):
        pass
    
    def process(self):
        pass
dataset = YooChooseBinaryDataset('C:\\Users\\User\\yoochoose-data\\discrepancy-capa-snet-nw-capa-averaging-smooth-norm1_3-adsci.dataset')

In [2]:
y = []
for i in range(len(dataset)):
    y.append(dataset[i].y)

In [5]:
from torch.nn import Sequential, Linear, ReLU, Parameter
import torch.nn.functional as F
from torch.nn.init import uniform_,ones_,xavier_normal_
from torch_geometric.data import DataLoader
from torch.nn import BatchNorm1d,Dropout
from timeit import default_timer as timer
from torch_geometric.nn import  global_mean_pool, TopKPooling,GlobalAttention,GINConv,GATConv,ChebConv,GCNConv,ARMAConv
from torch_geometric.utils import to_dense_batch
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
import warnings
from torch_geometric.utils import softmax
from typing import Any

warnings.filterwarnings("ignore")

def reset(value: Any):
    if hasattr(value, 'reset_parameters'):
        value.reset_parameters()
    else:
        for child in value.children() if hasattr(value, 'children') else []:
            reset(child)

class MeanAttention(torch.nn.Module):
    def __init__(self, gate_nn):
        super().__init__()
        
        self.gate_nn = gate_nn
        
        self.reset_parameters()
        
    def reset_parameters(self):
        reset(self.gate_nn)
        
    def forward(self, x):
        x_m = torch.unsqueeze(self.gate_nn(torch.mean(x,1)),1)
        attn = torch.softmax(torch.sum(x*x_m,2),1)
        return torch.sum(torch.unsqueeze(attn,2)*x,1)


class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        num_features = 148
        #num_features = 23
        dim = 32
        #nn1 = Sequential(Linear(num_features, dim), ReLU(), Linear(dim, dim))
        #self.conv1 = GINConv(nn1)
        #self.conv1 = GCNConv(num_features,dim)
        #self.conv2 = GCNConv(dim,dim)
        self.conv1 = ChebConv(num_features,dim,2)
        self.conv2 = ChebConv(dim,dim,2)
        #self.conv2 = GATConv(dim,dim)
        #self.conv2 = GATConv(dim,dim)
        #nn2 = Sequential(Linear(dim, dim), ReLU(), Linear(dim, dim))
        #self.conv2 = GINConv(nn2)
        #self.norm = PairNorm(1)
        #nn3 =  Sequential(Linear(dim, 1))
        #self.pool = TopKPooling(32,0.5)
        #self.pool = GlobalAttention(nn3)
        #nn1 = Sequential(Linear(32,32))
        #self.pool = MeanAttention(nn1)
        
    def forward_(self, x, edge_index):
        #x = F.relu(self.conv1(x, edge_index))
        #x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv1(x, edge_index))
        #x = self.norm(x,batch)
        x = F.relu(self.conv2(x, edge_index))
        #x = self.conv2(x, edge_index)
        #return torch.cat((x1,x2),dim=-1)
        return x

           
    def forward(self, x_s, edge_index_s, x_t, edge_index_t, x_s_batch, x_t_batch):
        x_s = self.forward_(x_s,edge_index_s)
        x_t = self.forward_(x_t,edge_index_t)
        x_s = global_mean_pool(x_s, x_s_batch)
        x_t = global_mean_pool(x_t, x_t_batch)
        #x_s = F.normalize(x_s)
        #x_t = F.normalize(x_t)
        scale_factor = 32**-0.5
        #scale_factor = 1
        ##Mean Attention
        #x_s,_ = to_dense_batch(x_s, x_s_batch)
        #x_t,_ = to_dense_batch(x_t, x_t_batch)
        #x_s = self.pool(x_s)
        #x_t = self.pool(x_t)
        dot = x_s*x_t*scale_factor
        #dot = torch.sum(dot,2)
        return torch.sum(dot,1)

def train(epoch):
    model.train()

    #if epoch == 51:
        #for param_group in optimizer.param_groups:
            #param_group['lr'] = 0.5 * param_group['lr']
    loss_all = 0
    for data in data_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data.x_s, data.edge_index_s, data.x_t, data.edge_index_t, data.x_s_batch, data.x_t_batch)
        var_1 = torch.var(output[torch.where(data.y==1)], unbiased=False)
        var_2 = torch.var(output[torch.where(data.y==0)], unbiased=False)
        mean_1 = torch.mean(output[torch.where(data.y==1)])
        mean_2 = torch.mean(output[torch.where(data.y==0)])
        loss = 0.1*var_1 + 0.1*var_2 + F.relu(0.5-(mean_1-mean_2))
        #for param in model.parameters():
            #loss += torch.sum(torch.abs(param))*1e-5
        #loss = 0.1*var_1 + 0.1*var_2 + F.relu(0.3 - torch.abs(mean_2-mean_1))
        #loss = 0.3*(torch.max(torch.abs(output[torch.where(data.y==1)]-mean_1)) + torch.max(torch.abs(output[torch.where(data.y==0)]-mean_2))) + F.relu(5-(mean_2-mean_1))
        loss.backward()
        loss_all += loss.item() * data.num_graphs
        optimizer.step()
    return loss_all / len(dataset)


def test(loader1,loader2):
    model.eval()
    correct = 0
    for data in loader1 :        
        data = data.to(device)
        output = model(data.x_s, data.edge_index_s, data.x_t, data.edge_index_t, data.x_s_batch, data.x_t_batch)
        model_1 = KNeighborsClassifier(n_neighbors=7)
        model_1.fit(output.detach().cpu().numpy().reshape(-1, 1),data.y.detach().cpu().numpy())
        #model_1 = LinearSVC()
        #model_1.fit(output.detach().cpu().numpy().reshape(-1, 1),data.y.detach().cpu().numpy())
    for data in loader2 :
        data = data.to(device)
        output = model(data.x_s, data.edge_index_s, data.x_t, data.edge_index_t, data.x_s_batch, data.x_t_batch)
        correct = model_1.score(output.detach().cpu().numpy().reshape(-1, 1),data.y.detach().cpu().numpy())
        correct = model_1.score(output.detach().cpu().numpy().reshape(-1, 1),data.y.detach().cpu().numpy())
    del model_1
    return correct

batch_size= 21

test_num = 20
results = np.zeros([test_num])
f1_arr = np.zeros([test_num])
patient_results = np.zeros([len(dataset),test_num])

#start=timer()
for t in range(0,test_num):
    
    acc = 0
    for i in range(0,len(dataset)) :
    #for i in range(2,3) :
        train_indices = np.delete(np.arange(len(dataset)),i).tolist() 
        test_indices = [i]
        dataset_train =  torch.utils.data.Subset(dataset, train_indices)
        dataset_test = torch.utils.data.Subset(dataset, test_indices)

        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        #device = torch.device('cpu')
        print(device)
        model = Net().to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)


        for epoch in range(1, 101):
            data_loader = DataLoader(dataset_train, batch_size=batch_size,shuffle=True,follow_batch=['x_s', 'x_t'])
            data_loader_test = DataLoader(dataset_test, batch_size=1,follow_batch=['x_s', 'x_t'])
            train_loss = train(epoch)
            #train_acc,_ = test(data_loader)
            #train_acc = train_acc/(len(dataset)-1)
            #test_acc,test_loss = test(data_loader_test)
            #print('Epoch: {:03d}, Train Loss: {:.7f}, '
                  #.format(epoch, train_loss,))
        data_all = DataLoader(dataset_train, batch_size=len(dataset)-1 ,shuffle=True,follow_batch=['x_s', 'x_t'])
        test_acc = test(data_all,data_loader_test)
        patient_results[i,t] = test_acc
        f1_arr[t] = f1_calculation(y,patient_results[:,t])
        acc = acc + test_acc
        #print(acc)
        del model
    acc = acc/len(dataset) 
    print(acc)
    results[t] = acc
#end = timer()
#print(end - start)

In [4]:
print("7NN Discrepancy Detector, Model:ReLU ChebyGCN(148->32)+ ReLU ChebyGCN(32->32) + MeanPool, K=2, Mean Loss(|AD-SCI|) ")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

7NN Discrepancy Detector, Model:ReLU ChebyGCN(148->32)+ ReLU ChebyGCN(32->32) + MeanPool, K=2, Mean Loss(|AD-SCI|) 
[0.85714286 0.88095238 0.85714286 0.85714286 0.80952381 0.85714286
 0.85714286 0.9047619  0.83333333 0.85714286 0.83333333 0.80952381
 0.83333333 0.83333333 0.80952381 0.83333333 0.85714286 0.83333333
 0.85714286 0.83333333]
Accuracy : 0.8452 ± 0.0232
F1 Score : 0.8075 ± 0.0296


In [6]:
print("7NN Discrepancy Detector, Model:ReLU ChebyGCN(148->32)+ ReLU ChebyGCN(32->32) + MeanPool, K=2, Mean Loss(AD-SCI) ")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

7NN Discrepancy Detector, Model:ReLU ChebyGCN(148->32)+ ReLU ChebyGCN(32->32) + MeanPool, K=2, Mean Loss(AD-SCI) 
[0.80952381 0.78571429 0.85714286 0.80952381 0.80952381 0.83333333
 0.83333333 0.78571429 0.78571429 0.78571429 0.80952381 0.80952381
 0.78571429 0.80952381 0.83333333 0.80952381 0.83333333 0.78571429
 0.78571429 0.80952381]
Accuracy : 0.8083 ± 0.0206
F1 Score : 0.7550 ± 0.0252


In [5]:
print("7NN Discrepancy Detector, Model:ReLU ChebyGCN(148->32)+ ReLU ChebyGCN(32->32) + MeanPool, K=2")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

7NN Discrepancy Detector, Model:ReLU ChebyGCN(148->32)+ ReLU ChebyGCN(32->32) + MeanPool, K=2
[0.9047619  0.88095238 0.88095238 0.85714286 0.88095238 0.9047619
 0.9047619  0.88095238 0.9047619  0.88095238 0.9047619  0.88095238
 0.88095238 0.85714286 0.9047619  0.88095238 0.85714286 0.9047619
 0.9047619  0.85714286]
Accuracy : 0.8857 ± 0.0178
F1 Score : 0.8579 ± 0.0224


In [4]:
print("7NN Discrepancy Detector, Model:ReLU ARMAGCN(148->32) + ReLU ARMAGCN(32->32) + MeanPool")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

7NN Discrepancy Detector, Model:ReLU ARMAGCN(148->32) + ReLU ARMAGCN(32->32) + MeanPool
[0.9047619  0.85714286 0.9047619  0.9047619  0.9047619  0.88095238
 0.88095238 0.88095238 0.88095238 0.83333333 0.83333333 0.9047619
 0.83333333 0.88095238 0.88095238 0.9047619  0.88095238 0.85714286
 0.9047619  0.88095238]
Accuracy : 0.8798 ± 0.0244
F1 Score : 0.8513 ± 0.0294


In [10]:
print("7NN Discrepancy Detector, Model:ReLU ChebyGCN(148->32) + MeanPool, K=2")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

7NN Discrepancy Detector, Model:ReLU ChebyGCN(148->32) + MeanPool, K=2
[0.85714286 0.85714286 0.85714286 0.83333333 0.88095238 0.85714286
 0.88095238 0.85714286 0.85714286 0.83333333 0.85714286 0.85714286
 0.85714286 0.85714286 0.85714286 0.85714286 0.85714286 0.85714286
 0.85714286 0.80952381]
Accuracy : 0.8548 ± 0.0149
F1 Score : 0.8194 ± 0.0189


In [8]:
print("7NN Discrepancy Detector, Model:ReLU ARMAGCN(148->32) + ReLU ARMAGCN(32->32) + MeanPool")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

7NN Discrepancy Detector, Model:ReLU ARMAGCN(148->32) + ReLU ARMAGCN(32->32) + MeanPool
[0.85714286 0.9047619  0.88095238 0.88095238 0.9047619  0.83333333
 0.83333333 0.88095238 0.88095238 0.88095238 0.83333333 0.85714286
 0.85714286 0.88095238 0.88095238 0.88095238 0.88095238 0.9047619
 0.85714286 0.85714286]
Accuracy : 0.8714 ± 0.0218
F1 Score : 0.8406 ± 0.0271


In [28]:
print("Rewired AD-SCI, 7NN Discrepancy Detector, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + MeanPool")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

Rewired AD-SCI, 7NN Discrepancy Detector, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + MeanPool
[0.83333333 0.80952381 0.83333333 0.80952381 0.83333333]
Accuracy : 0.8238 ± 0.0117
F1 Score : 0.7911 ± 0.0109


In [22]:
print("SCI-MCI, 7NN Discrepancy Detector, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + MeanPool")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

SCI-MCI, 7NN Discrepancy Detector, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + MeanPool
[0.7        0.71428571 0.67142857 0.71428571 0.7       ]
Accuracy : 0.7000 ± 0.0156
F1 Score : 0.7827 ± 0.0093


In [6]:
print("RANDOMİZED SCI-MCI, 7NN Discrepancy Detector, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + MeanPool")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

 RANDOMİZED SCI-MCI, 7NN Discrepancy Detector, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + MeanPool
[0.58571429 0.57142857 0.62857143 0.57142857 0.64285714]
Accuracy : 0.6000 ± 0.0300
F1 Score : 0.7264 ± 0.0225


In [6]:
print(" AD-MCI, 7NN Discrepancy Detector, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + MeanPool")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

 AD-MCI, 7NN Discrepancy Detector, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + MeanPool
[0.78125  0.78125  0.765625 0.78125  0.765625]
Accuracy : 0.7750 ± 0.0077
F1 Score : 0.5131 ± 0.0248


In [14]:
print(" RANDOMİZED AD-MCI, 7NN Discrepancy Detector, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + MeanPool")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

 RANDOMİZED AD-MCI, 7NN Discrepancy Detector, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + MeanPool
[0.625    0.640625 0.640625 0.625    0.609375]
Accuracy : 0.6281 ± 0.0117
F1 Score : 0.1308 ± 0.0272


In [11]:
print(" STRATIFIED LABELS 7NN Discrepancy Detector, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + MeanPool")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

 STRATIFIED LABELS 7NN Discrepancy Detector, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + MeanPool
[0.52542373 0.55932203 0.42372881 0.52542373 0.49152542 0.49152542
 0.50847458 0.47457627 0.54237288 0.49152542 0.44067797 0.50847458
 0.47457627 0.49152542 0.44067797 0.45762712 0.52542373 0.50847458
 0.50847458 0.55932203]
Accuracy : 0.4975 ± 0.0369
F1 Score : 0.4757 ± 0.0447


In [8]:
print(" RANDOMIZED LABELS 7NN Discrepancy Detector, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + MeanPool")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

 RANDOMIZED LABELS 7NN Discrepancy Detector, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + MeanPool
[0.33333333 0.4047619  0.4047619  0.38095238 0.4047619  0.33333333
 0.42857143 0.4047619  0.4047619  0.35714286 0.33333333 0.52380952
 0.4047619  0.42857143 0.4047619  0.45238095 0.38095238 0.38095238
 0.35714286 0.38095238]
Accuracy : 0.3952 ± 0.0435
F1 Score : 0.1262 ± 0.0637


In [9]:
print("7NN Discrepancy Detector, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + MeanPool")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

7NN Discrepancy Detector, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + MeanPool
[0.80952381 0.80952381 0.83333333 0.76190476 0.78571429 0.78571429
 0.80952381 0.78571429 0.78571429 0.80952381 0.83333333 0.83333333
 0.80952381 0.83333333 0.78571429 0.80952381 0.85714286 0.83333333
 0.78571429 0.76190476]
Accuracy : 0.8060 ± 0.0253
F1 Score : 0.7697 ± 0.0279


In [5]:
print("7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + MeanPool with Jumping Knowledge")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + MeanPool with Jumping Knowledge
[0.9047619  0.85714286 0.88095238 0.85714286 0.88095238 0.85714286
 0.85714286 0.85714286 0.88095238 0.85714286 0.85714286 0.85714286
 0.85714286 0.88095238 0.85714286 0.88095238 0.88095238 0.83333333
 0.9047619  0.83333333]
Accuracy : 0.8667 ± 0.0190
F1 Score : 0.8354 ± 0.0223


In [4]:
print("7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + MeanPool")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + MeanPool
[0.85714286 0.88095238 0.88095238 0.85714286 0.88095238 0.85714286
 0.88095238 0.85714286 0.83333333 0.85714286 0.83333333 0.85714286
 0.88095238 0.88095238 0.88095238 0.83333333 0.85714286 0.85714286
 0.9047619  0.88095238]
Accuracy : 0.8655 ± 0.0189
F1 Score : 0.8323 ± 0.0215


In [28]:
print("7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + MeanPool")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + MeanPool
[0.85714286 0.88095238 0.85714286 0.85714286 0.85714286 0.85714286
 0.88095238 0.83333333 0.88095238 0.9047619  0.88095238 0.88095238
 0.88095238 0.85714286 0.88095238 0.88095238 0.88095238 0.85714286
 0.88095238 0.88095238]
Accuracy : 0.8714 ± 0.0158
F1 Score : 0.8362 ± 0.0210


In [32]:
print("7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + MeanPool (0.2var and 32scalingfactor)")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + MeanPool (0.2var and 32scalingfactor)
[0.85714286 0.88095238 0.88095238 0.88095238 0.88095238 0.88095238
 0.85714286 0.88095238 0.88095238 0.88095238 0.85714286 0.88095238
 0.88095238 0.88095238 0.9047619  0.88095238 0.88095238 0.88095238
 0.83333333 0.88095238]
Accuracy : 0.8762 ± 0.0143
F1 Score : 0.8432 ± 0.0195


In [37]:
print("7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->64) + ReLU GCN(64->64) + MeanPool (0.2var,32sf)")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->64) + ReLU GCN(64->64) + MeanPool (0.2var,32sf)
[0.88095238 0.85714286 0.85714286 0.85714286 0.80952381 0.88095238
 0.88095238 0.85714286 0.88095238 0.88095238 0.88095238 0.88095238
 0.85714286 0.88095238 0.85714286 0.85714286 0.85714286 0.88095238
 0.88095238 0.85714286]
Accuracy : 0.8667 ± 0.0175
F1 Score : 0.8337 ± 0.0177


In [4]:
print("7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + MeanPool, Mean Loss(AD-SCI)")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + MeanPool, Mean Loss(AD-SCI)
[0.80952381 0.80952381 0.78571429 0.83333333 0.85714286 0.78571429
 0.80952381 0.80952381 0.80952381 0.80952381 0.80952381 0.78571429
 0.80952381 0.83333333 0.85714286 0.78571429 0.83333333 0.80952381
 0.83333333 0.80952381]
Accuracy : 0.8143 ± 0.0208
F1 Score : 0.7722 ± 0.0281


In [12]:
print("7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + MeanPool (AD-MCI)")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + MeanPool (AD-MCI)
[0.8125 0.8125]
Accuracy : 0.8125 ± 0.0000
F1 Score : 0.6000 ± 0.0000


In [79]:
print("7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + batchsize14betterTraining")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + batchsize14betterTraining
[0.83333333 0.80952381 0.85714286 0.88095238 0.85714286 0.83333333
 0.88095238 0.88095238 0.83333333 0.85714286 0.85714286 0.85714286
 0.78571429 0.80952381 0.85714286 0.80952381 0.85714286 0.83333333
 0.85714286 0.88095238]
Accuracy : 0.8464 ± 0.0266
F1 Score : 0.8140 ± 0.0266


In [83]:
print("7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + batchsize14epoch150")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + batchsize14epoch150
[0.80952381 0.80952381 0.83333333 0.76190476 0.78571429 0.78571429
 0.78571429 0.83333333 0.83333333 0.83333333 0.80952381 0.80952381
 0.85714286 0.80952381 0.80952381 0.80952381 0.85714286 0.80952381
 0.76190476 0.80952381]
Accuracy : 0.8107 ± 0.0255
F1 Score : 0.7697 ± 0.0317


In [87]:
print("7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + batchsize14sf16")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + ReLU GCN(32->32) + batchsize14sf16
[0.83333333 0.83333333 0.88095238 0.83333333 0.88095238 0.83333333
 0.88095238 0.85714286 0.85714286 0.83333333 0.85714286 0.85714286
 0.88095238 0.83333333 0.85714286 0.85714286 0.83333333 0.80952381
 0.85714286 0.83333333]
Accuracy : 0.8500 ± 0.0201
F1 Score : 0.8177 ± 0.0212


In [102]:
print("7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + GCN(32->32) + sf8margin5")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + GCN(32->32) + sf8margin5
[0.85714286 0.85714286 0.85714286 0.88095238 0.85714286 0.83333333
 0.83333333 0.85714286 0.85714286 0.85714286 0.85714286 0.85714286
 0.85714286 0.80952381 0.88095238 0.88095238 0.9047619  0.83333333
 0.88095238 0.85714286]
Accuracy : 0.8583 ± 0.0206
F1 Score : 0.8266 ± 0.0225


In [4]:
print("7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + GCN(32->32) + sf8margin3var0.2")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + GCN(32->32) + sf8margin3var0.2
[0.83333333 0.80952381 0.85714286 0.83333333 0.83333333 0.85714286
 0.83333333 0.83333333 0.88095238 0.85714286 0.83333333 0.85714286
 0.85714286 0.83333333 0.88095238 0.85714286 0.85714286 0.83333333
 0.83333333 0.83333333]
Accuracy : 0.8452 ± 0.0177
F1 Score : 0.8114 ± 0.0224


In [12]:
print("7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + GCN(32->32) ")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + GCN(32->32) 
[0.85714286 0.83333333 0.85714286 0.88095238 0.85714286 0.85714286
 0.85714286 0.85714286 0.83333333 0.85714286 0.9047619  0.88095238
 0.83333333 0.83333333 0.83333333 0.85714286 0.83333333 0.85714286
 0.85714286 0.83333333]
Accuracy : 0.8536 ± 0.0189
F1 Score : 0.8198 ± 0.0235


In [16]:
print("7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + ReLU GCN(32->32) margin3")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + ReLU GCN(32->32) margin3
[0.80952381 0.85714286 0.85714286 0.85714286 0.88095238 0.83333333
 0.83333333 0.88095238 0.85714286 0.85714286 0.83333333 0.88095238
 0.88095238 0.85714286 0.85714286 0.88095238 0.85714286 0.9047619
 0.88095238 0.88095238]
Accuracy : 0.8619 ± 0.0221
F1 Score : 0.8292 ± 0.0249


In [42]:
print("7NN Discrepancy Detector for Smooth1.3, Model:ReLU GIN(148->32) + ReLU GIN(32->32) margin0.5")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

7NN Discrepancy Detector for Smooth1.3, Model:ReLU GIN(148->32) + ReLU GIN(32->32) margin0.5
[0.69047619 0.73809524 0.66666667 0.76190476 0.66666667 0.73809524
 0.76190476 0.71428571 0.66666667 0.71428571 0.76190476 0.73809524
 0.78571429 0.80952381 0.78571429 0.78571429 0.71428571 0.76190476
 0.76190476 0.80952381]
Accuracy : 0.7417 ± 0.0441
F1 Score : 0.7152 ± 0.0442


In [4]:
print("7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + ReLU GCN(32->32) margin0.5 RANDOM SNET")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + ReLU GCN(32->32) margin0.5 RANDOM SNET
[0.76190476 0.73809524 0.78571429 0.73809524 0.76190476 0.76190476
 0.76190476 0.80952381 0.76190476 0.83333333 0.71428571 0.71428571
 0.78571429 0.76190476 0.80952381 0.78571429 0.78571429 0.80952381
 0.78571429 0.76190476]
Accuracy : 0.7714 ± 0.0305
F1 Score : 0.7411 ± 0.0293


In [24]:
print("7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + ReLU GCN(32->32) margin0.5 RANDOM LNET")
print(results)
print(f'Accuracy : {np.mean(results):.4f} ± {np.std(results):.4f}')
print(f'F1 Score : {np.mean(f1_arr):.4f} ± {np.std(f1_arr):.4f}')

7NN Discrepancy Detector for Smooth1.3, Model:ReLU GCN(148->32) + ReLU GCN(32->32) margin0.5 RANDOM LNET
[0.80952381 0.78571429 0.76190476 0.78571429 0.76190476 0.76190476
 0.78571429 0.76190476 0.78571429 0.78571429 0.78571429 0.80952381
 0.76190476 0.76190476 0.80952381 0.80952381 0.78571429 0.78571429
 0.76190476 0.83333333]
Accuracy : 0.7845 ± 0.0206
F1 Score : 0.7395 ± 0.0309
